In [1]:
import pandas as pd

In [4]:
unique_kalpiot = pd.read_csv('unique_kalpiot_basic_google_geocoding.csv', index_col=0)

In [19]:
# Missing Lat
unique_kalpiot.loc[unique_kalpiot['Latitude'].isna()]

,town_symbol,town_name,location_symbol,ballot_address,ballot_location,bldg_nubmer,street,geocode_address,Latitude,Longitude
1421,4006,קשת,1,קשת,מועדון,NaN,קשת,"קשת, קשת",NaN,NaN
1564,1230,לבנים,2,לבנים,מועדון לחבר,NaN,לבנים,"לבנים, לבנים",NaN,NaN
1724,141,ברק,1,ברק,בית העם (צמוד למזכירות),NaN,ברק,"ברק, ברק",NaN,NaN
1742,146,דבורה,1,דבורה,מזכירות,NaN,דבורה,"דבורה, דבורה",NaN,NaN
1781,1110,חבר,1,חבר,מזכירות,NaN,חבר,"חבר, חבר",NaN,NaN
1973,1413,מחנה יהודית,1,מחנה יהודית,מועדון חוגים,NaN,מחנה יהודית,"מחנה יהודית, מחנה יהודית",NaN,NaN
2186,526,סולם,1,סולם,"בי""ס יסודי ממלכתי",NaN,סולם,"סולם, סולם",NaN,NaN
2452,1208,הילה,1,הילה,מזכירות,NaN,הילה,"הילה, הילה",NaN,NaN
4071,3717,חרמש,1,חרמש,מזכירות,NaN,חרמש,"חרמש, חרמש",NaN,NaN
6060,848,בית רבן,1,בית רבן,מכללת גבעת וושינגטון,NaN,בית רבן,"בית רבן, בית רבן",NaN,NaN


In [18]:
i = 1282
unique_kalpiot.loc[i, 'Latitude']  = 33.24768 
unique_kalpiot.loc[i, 'Longitude'] = 35.59774
